In [1]:
import pandas as pd
import numpy as np
import glob
import time

%pylab inline
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import matplotlib.pylab as pylab

from matplotlib.backends.backend_pdf import PdfPages

Populating the interactive namespace from numpy and matplotlib


In [2]:
#define function to load up datafiles
def loadData( fileName, skipR = 0 ):

    files = glob.glob( fileName )  
    df = pd.DataFrame();
    for f in files:
        #subID = f.split('/')[1][0:-4]
        thisSub = pd.read_csv( f,sep='\t', skiprows=skipR)
        #thisSub['subID'] = subID
        df = pd.concat( [df, thisSub ], ignore_index=True)        
    return df

# return true if relevant ( return nan if nan)
def isRel( AOI ):
    if np.isnan( AOI ):
        return np.nan    
    if AOI in [1,2,3,4]: # any of first 4 AOIs is relevant for the spores! 
        return True
    else:
        return False



In [3]:
#load up mix files ( old sound condition for cogsci, leave alone)
# outfile = 'mixDataByTrial.txt'
# files = 'DataMix/*.txt'
# rawColumns = [ 'subid', 'TRAIL_INDEX', 'trialTime','RESPONSE', 'RT', 'stimulus','target_auditory','target_visual', 'test_auditory','test_visual'] #, 'trial'

# #the visual condition ( old vis condition for cogsci, leave alone)
# outfile = 'visDataByTrial.txt'
# files = 'DataVis/*.txt'
# rawColumns = [ 'subid',  'TRAIL_INDEX', 'trialTime','RESPONSE', 'RT', 'stimulus','target', 'test'] #,'trial'

# # aud only files (bad condition, old sound condition for cogsci, leave alone)
# outfile = 'audDataByTrial.txt'
# files = 'DataAud/*.txt'
# rawColumns = [ 'subid', 'TRAIL_INDEX','RESPONSE', 'RT', 'stimulus','target_auditory', 'test_auditory'] # , 'trial'

# # Chris test of auditory only
# outfile = 'NewSoundAudByTrialCR.txt'
# files = 'DataNewSoundAudCR/*.txt'
# rawColumns = [ 'subid', 'TRAIL_INDEX', 'trialTime','RESPONSE', 'RT', 'stimulus','target_auditory','target_visual', 'test_auditory','test_visual'] # , 'trial'




condList = []


# # spore study (Adult)
# condList.append( {                   
#             'outfile':'SporeTrainByTrial.txt',
#             'files': 'DataSporeTrain/*.txt',
#             'rawColumns' : [ 'subid', 'TRAIL_INDEX', 'block', 'trialTime','RESPONSE', 'KEYPRESS', 'correct_response', 'RTEstimate','cond', 'type', 'auditory','visual'] })#'trial', 'RT'

# condList.append( {                   
#             'outfile':'SporeTestByTrial.txt',
#             'files': 'DataSporeTest/*.txt',
#             'rawColumns' : [ 'subid', 'TRAIL_INDEX', 'block', 'trialTime','RESPONSE', 'KEYPRESS', 'correct_response', 'RTEstimate', 'cond', 'type', 'trialtype','visual'] })#'trial', 'RT'



# # --------------------------------------------------
# # spore study (Kids)
# condList.append( {                   
#             'outfile':'SporeTrainByTrialKids.txt',
#             'files': 'DataSporeTrainKids/*.txt',
#             'rawColumns' : [ 'subid', 'TRAIL_INDEX', 'block', 'trialTime','RESPONSE', 'KEYPRESS', 'correct_response', 'RTEstimate','cond', 'type', 'auditory','visual'] })#'trial', 'RT'

# condList.append( {                   
#             'outfile':'SporeTestByTrialKids.txt',
#             'files': 'DataSporeTestKids/*.txt',
#             'rawColumns' : [ 'subid', 'TRAIL_INDEX', 'block', 'trialTime','RESPONSE', 'KEYPRESS', 'correct_response', 'RTEstimate', 'cond', 'type', 'trialtype','visual'] })#'trial', 'RT'


# # --------------------------------------------------
# spore study (Adult)
condList.append( {                   
            'outfile':'SporeTrainByTrialNewAdult.txt',
            'files': 'DataSporeTrainNewAdult/*.txt',
            'rawColumns' : [ 'subid', 'TRAIL_INDEX', 'block', 'trialTime','RESPONSE', 'KEYPRESS', 'correct_response', 'RTEstimate','cond', 'type', 'auditory','visual'] })#'trial', 'RT'

condList.append( {                   
            'outfile':'SporeTestByTrialNewAdult.txt',
            'files': 'DataSporeTestNewAdult/*.txt',
            'rawColumns' : [ 'subid', 'TRAIL_INDEX', 'block', 'trialTime','RESPONSE', 'KEYPRESS', 'correct_response', 'RTEstimate', 'cond', 'type', 'trialtype','visual'] })#'trial', 'RT'







In [4]:
# df.columns

In [5]:
# define a bunch of functions that extract the basic variables

def latFix( someTrials, startTime):
    # latency to first fixation
    if len(someTrials) > 0:
        lat = someTrials.fixStart.min() - startTime
        return lat
    else:
        return np.nan

def meanFixDur(someTrials ):
    # calc mean fixation duration
    return someTrials.fixDuration.mean()

def meanPupSize(someTrials ):
    # calc mean fixation duration
    return someTrials.pupil.mean()

def numFixSac( someTrials):
    # calculate number fixations and number saccades
    numFix =  someTrials['fixStart'].count()
    numSac =  someTrials['sacAmp'].count()
    return (numFix, numSac)

#
def sacAmp( someTrials):
    return someTrials.sacAmp.sum()

# return nan if no looks, or proportion look at relevant
def propRel( someTrials ):    
    relLook = someTrials[ someTrials.relFix == True].fixDuration.sum()    
    allLook = someTrials['fixDuration'].sum()
    if allLook == 0:
        return np.nan
    else:
        return relLook/float(allLook)

def firstRel( someTrials ):
    fix = someTrials[ someTrials.fixDuration >= 0 ]
    if len(fix >0 ):
        return fix.relFix.iloc[0]

# lat = latFix( target, targOn[0])
# latRel = latFix( target[ target.relFix == True ], targOn[0])

# mDur = meanFixDur( target)
# pup = meanPupSize( target)
# (nFix, nSac) = numFixSac( target)
# sAmp = sacAmp( target)

# relProp = propRel( target )

# print lat,latRel, mDur, pup, (nFix, nSac), sAmp, relProp

In [6]:
# Loop through trials and calculate measures


for cond in condList:
    
    # get file info for this condition
    outfile = cond[ 'outfile']
    files = cond[ 'files']
    rawColumns = cond[ 'rawColumns'] 
    
    # load this condition, define relevant fix/saccades
    df = loadData( files )
    df[ 'relFix' ] = df[ 'fixAois'].apply( lambda(x): isRel(x))
    df[ 'relSac' ] = df[ 'sacAois'].apply( lambda(x): isRel(x))    
    
    # create dataframe for each condition separately
    dfByTrial = pd.DataFrame();
    subs = df.subid.unique()
    for s in subs:
        trials = sort(df[ (df.subid == s)].TRAIL_INDEX.unique()) 
        
        for t in trials:
            #summarize this trial dv's
            thisTrialRaw = df[ (df.subid == s) & (df.TRAIL_INDEX == t)]   # raw dv's              
            thisTrial = thisTrialRaw[ rawColumns ] # grab wanted columns
            thisTrial = thisTrial[:1] # just first row
            
            #-------------
            # get target and test fixations/saccades\
            if 'trialTime' in thisTrial.columns:
                targOn = thisTrial.trialTime.as_matrix()
                # from display on -> 1000 ms
                target = thisTrialRaw #[ (thisTrialRaw.fixStart < targOn[0]) 
                                      #| (thisTrialRaw.sacStartTime < targOn[0] ) ]        
                
                # from 2000ms after display on
                #test = thisTrialRaw[ (thisTrialRaw.fixStart > targOn[0]+2000)  
                #                    | (thisTrialRaw.sacStartTime > targOn[0]+2000  ) ]
                #-------------            
                # now calculate other measures (on target  )
                thisTrial['trgLatFix'] = latFix( target, targOn[0])                
                thisTrial['trgLatRelFix'] = latFix( target[ target.relFix == True ], targOn[0])
                thisTrial['trgMDur'] = meanFixDur( target)
                thisTrial['trgMpup'] = meanPupSize( target)
                (thisTrial['trgNFix'], thisTrial['trgNSac']) = numFixSac( target)
                thisTrial['trgSAmp'] = sacAmp( target)
                thisTrial['trgRelProp'] = propRel( target )
                thisTrial['trgFirstFixRel'] =  firstRel( target )
    
                '''
                # now calculate other measures (on test )
                thisTrial['tstLatFix'] = latFix( test, targOn[0]+2000)                
                thisTrial['tstLatRelFix'] = latFix( test[ test.relFix == True ], targOn[0]+2000)
                thisTrial['tstMDur'] = meanFixDur( test)
                thisTrial['tstMpup'] = meanPupSize( test)
                (thisTrial['tstNFix'], thisTrial['tstNSac']) = numFixSac( test)
                thisTrial['tstSAmp'] = sacAmp( test)
                thisTrial['tstRelProp'] = propRel( test )        
                thisTrial['tstFirstFixRel'] =  firstRel( test )
                '''
        
                # add True if warning beep ( if participant looked away)
    #             print thisTrialRaw[ ['TRAIL_INDEX', 'lastMessage', 'trialTime'  ] ]            
                thisTrial['warningBeepOn'] = thisTrialRaw.lastMessage.dropna().str.contains( 'REMOTE_WARNING_AUDIO').any()
    #             print thisTrial['warningBeepOn']
                
            # concatenate to big list
            dfByTrial = pd.concat( [dfByTrial, thisTrial ], ignore_index=True) 
    
    dfByTrial['logRT'] = np.log(dfByTrial.RTEstimate)  
    dfByTrial.to_csv(outfile ,sep='\t', mode='w', index=False)
    
    # dfByTrial

In [7]:
# thisTrial.trialTime.as_matrix()

In [8]:
# dfByTrial.type.unique()
# dfByTrial
# target

In [9]:
# dfByTrial.groupby(['subid', 'type']).head(5)